In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline


In [27]:
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-11-02 09:27:11--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-11-02 09:27:12--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv.4’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.02s   

2024-11-02 09:27:12 (17.1 MB/s) - ‘jamb_exam_resu

In [28]:
df = pd.read_csv("jamb_exam_results.csv")

In [29]:
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


In [33]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

Preparation:

Remove the student_id column.
Fill missing values with zeros.
Do train/validation/test split with 60%/20%/20% distribution.
Use the train_test_split function and set the random_state parameter to 1.
Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [34]:
df.columns

Index(['jamb_score', 'study_hours_per_week', 'attendance_rate',
       'teacher_quality', 'distance_to_school', 'school_type',
       'school_location', 'extra_tutorials', 'access_to_learning_materials',
       'parent_involvement', 'it_knowledge', 'student_id', 'age', 'gender',
       'socioeconomic_status', 'parent_education_level',
       'assignments_completed'],
      dtype='object')

In [35]:
df = df.drop(columns=['student_id'])

In [36]:
df.isnull().sum()

jamb_score                        0
study_hours_per_week              0
attendance_rate                   0
teacher_quality                   0
distance_to_school                0
school_type                       0
school_location                   0
extra_tutorials                   0
access_to_learning_materials      0
parent_involvement                0
it_knowledge                      0
age                               0
gender                            0
socioeconomic_status              0
parent_education_level          891
assignments_completed             0
dtype: int64

In [37]:
df = df.fillna(0)

In [38]:
df.isnull().sum()

jamb_score                      0
study_hours_per_week            0
attendance_rate                 0
teacher_quality                 0
distance_to_school              0
school_type                     0
school_location                 0
extra_tutorials                 0
access_to_learning_materials    0
parent_involvement              0
it_knowledge                    0
age                             0
gender                          0
socioeconomic_status            0
parent_education_level          0
assignments_completed           0
dtype: int64

In [39]:
from sklearn.model_selection import train_test_split
#Do train/validation/test split with 60%/20%/20% distribution. Use the train_test_split function and set the random_state parameter to 1.
X = df.drop(columns=['jamb_score'])
y =df['jamb_score']
X_train, X_temp, y_train,y_temp = train_test_split(X, y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

#Use DictVectorizer(sparse=True) to turn the dataframes into matrices.
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)

X_train_dict = X_train.to_dict(orient='records')
X_val_dict = X_val.to_dict(orient='records')
X_test_dict = X_test.to_dict(orient='records')

X_train_matrix = dv.fit_transform(X_train_dict)

X_val_matrix = dv.transform(X_val_dict)
X_test_matrix = dv.transform(X_test_dict)


In [42]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train_matrix,y_train)

dt.tree_.feature
#split_feature_index = dt.tree_.feature[0]
#print(split_feature_index)

array([27, -2, -2], dtype=int64)

In [41]:
split_feature_name = dv.feature_names_[split_feature_index]
print(split_feature_name)

study_hours_per_week


In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
rf  = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)

rf.fit(X_train_matrix, y_train)

y_val_pred = rf.predict(X_val_matrix)

rmse = (root_mean_squared_error(y_val, y_val_pred))

print(f"the value of RMSE on this model is {rmse:.2f}")


the value of RMSE on this model is 43.16


In [49]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error


rmse_dict = {}
# loop for rmse values for n values ranging from 10 to 200 with step of 10
for n in range(10,201,10):
    rf  = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train_matrix, y_train)
    y_val_pred = rf.predict(X_val_matrix)
    rmse = (root_mean_squared_error(y_val, y_val_pred))
    rmse_dict[n] = rmse
    print(f" RMSE value is {rmse:.3f} for estimator value of {n}")
    
last_rmse = None

for n, rmse in rmse_dict.items():
    rounded_rmse = round(rmse,3)
    if last_rmse is not None and rounded_rmse==last_rmse:
        print(f"RMSE stops improving significantly after n_estimators={n}")
        break
    last_rmse = rounded_rmse
        
    



 RMSE value is 43.157759 for estimator value of 10
 RMSE value is 41.790440 for estimator value of 20
 RMSE value is 41.555818 for estimator value of 30
 RMSE value is 41.075632 for estimator value of 40
 RMSE value is 40.957157 for estimator value of 50
 RMSE value is 40.773685 for estimator value of 60
 RMSE value is 40.587806 for estimator value of 70
 RMSE value is 40.502704 for estimator value of 80
 RMSE value is 40.434922 for estimator value of 90
 RMSE value is 40.364910 for estimator value of 100
 RMSE value is 40.347525 for estimator value of 110
 RMSE value is 40.301918 for estimator value of 120
 RMSE value is 40.285789 for estimator value of 130
 RMSE value is 40.263461 for estimator value of 140
 RMSE value is 40.254264 for estimator value of 150
 RMSE value is 40.199666 for estimator value of 160
 RMSE value is 40.187326 for estimator value of 170
 RMSE value is 40.135963 for estimator value of 180
 RMSE value is 40.152166 for estimator value of 190
 RMSE value is 40.138